# Translating French to English with Pytorch

In [1]:
%matplotlib inline
import re, pickle, collections, bcolz, numpy as np, keras, sklearn, math, operator

Using TensorFlow backend.


In [2]:
import gensim

In [3]:
from gensim.models import word2vec

import torch, torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

In [4]:
path='data/datasets/fr-en-109-corpus/'
dpath = 'data/translate/'

## Prepare corpus

The French-English parallel corpus can be downloaded from http://www.statmt.org/wmt10/training-giga-fren.tar. It was created by Chris Callison-Burch, who crawled millions of web pages and then used 'a set of simple heuristics to transform French URLs onto English URLs (i.e. replacing "fr" with "en" and about 40 other hand-written rules), and assume that these documents are translations of each other'.

In [ ]:
fname=path+'giga-fren.release2.fixed'
en_fname = fname+'.en'
fr_fname = fname+'.fr'

To make this problem a little simpler so we can train our model more quickly, we'll just learn to translate questions that begin with 'Wh' (e.g. what, why, where which). Here are our regexps that filter the sentences we want.

In [ ]:
re_eq = re.compile('^(Wh[^?.!]+\?)')
re_fq = re.compile('^([^?.!]+\?)')

lines = ((re_eq.search(eq), re_fq.search(fq)) 
         for eq, fq in zip(open(en_fname), open(fr_fname)))

qs = [(e.group(), f.group()) for e,f in lines if e and f]; len(qs)

In [ ]:
qs[:6]

Because it takes a while to load the data, we save the results to make it easier to load in later.

In [ ]:
pickle.dump(qs, open(dpath+'fr-en-qs.pkl', 'wb'))

### Load saved french and english corpuses

In [5]:
qs = pickle.load(open(dpath+'fr-en-qs.pkl', 'rb'))

In [6]:
en_qs, fr_qs = zip(*qs)

Because we are translating at word level, we need to tokenize the text first. (Note that it is also possible to translate at character level, which doesn't require tokenizing.) There are many tokenizers available, but we found we got best results using these simple heuristics.

In [7]:
re_apos = re.compile(r"(\w)'s\b")         # make 's a separate word
re_mw_punc = re.compile(r"(\w[’'])(\w)")  # other ' in a word creates 2 words
re_punc = re.compile("([\"().,;:/_?!—])") # add spaces around punctuation
re_mult_space = re.compile(r"  *")        # replace multiple spaces with just one

def simple_toks(sent):
    sent = re_apos.sub(r"\1 's", sent)
    sent = re_mw_punc.sub(r"\1 \2", sent)
    sent = re_punc.sub(r" \1 ", sent).replace('-', ' ')
    sent = re_mult_space.sub(' ', sent)
    return sent.lower().split()

In [8]:
fr_qtoks = list(map(simple_toks, fr_qs)); fr_qtoks[:4]

[['qu’', 'est', 'ce', 'que', 'la', 'lumière', '?'],
 ['où', 'sommes', 'nous', '?'],
 ["d'", 'où', 'venons', 'nous', '?'],
 ['que', 'ferions', 'nous', 'sans', 'elle', '?']]

In [9]:
en_qtoks = list(map(simple_toks, en_qs)); en_qtoks[:4]

[['what', 'is', 'light', '?'],
 ['who', 'are', 'we', '?'],
 ['where', 'did', 'we', 'come', 'from', '?'],
 ['what', 'would', 'we', 'do', 'without', 'it', '?']]

In [10]:
simple_toks("Rachel's baby is cuter than other's.")

['rachel', "'s", 'baby', 'is', 'cuter', 'than', 'other', "'s", '.']

Special tokens used to pad the end of sentences, and to mark the start of a sentence.

In [11]:
PAD = 0; SOS = 1

Enumerate the unique words (*vocab*) in the corpus, and also create the reverse map (word->index). Then use this mapping to encode every sentence as a list of int indices.

In [12]:
def toks2ids(sents):
    voc_cnt = collections.Counter(t for sent in sents for t in sent)
    vocab = sorted(voc_cnt, key=voc_cnt.get, reverse=True)
    vocab.insert(PAD, "<PAD>")
    vocab.insert(SOS, "<SOS>")
    w2id = {w:i for i,w in enumerate(vocab)}
    ids = [[w2id[t] for t in sent] for sent in sents]
    return ids, vocab, w2id, voc_cnt

In [13]:
fr_ids, fr_vocab, fr_w2id, fr_counts = toks2ids(fr_qtoks)
en_ids, en_vocab, en_w2id, en_counts = toks2ids(en_qtoks)

## Word vectors

Stanford's GloVe word vectors can be downloaded from https://nlp.stanford.edu/projects/glove/ (in the code below we have preprocessed them into a bcolz array). We use these because each individual word has a single word vector, which is what we need for translation. Word2vec, on the other hand, often uses multi-word phrases.

In [14]:
def load_glove(loc):
    return (bcolz.open(loc+'.dat')[:],
        pickle.load(open(loc+'_words.pkl','rb'), encoding='latin1'),
        pickle.load(open(loc+'_idx.pkl','rb'), encoding='latin1'))

In [15]:
en_vecs, en_wv_word, en_wv_idx = load_glove('data/datasets/nlp/glove/results/6B.100d')
en_w2v = {w: en_vecs[en_wv_idx[w]] for w in en_wv_word}
n_en_vec, dim_en_vec = en_vecs.shape

In [16]:
en_w2v['king']

array([-0.32307 , -0.87616 ,  0.21977 ,  0.25268 ,  0.22976 ,  0.7388  ,
       -0.37954 , -0.35307 , -0.84369 , -1.1113  , -0.30266 ,  0.33178 ,
       -0.25113 ,  0.30448 , -0.077491, -0.89815 ,  0.092496, -1.1407  ,
       -0.58324 ,  0.66869 , -0.23122 , -0.95855 ,  0.28262 , -0.078848,
        0.75315 ,  0.26584 ,  0.3422  , -0.33949 ,  0.95608 ,  0.065641,
        0.45747 ,  0.39835 ,  0.57965 ,  0.39267 , -0.21851 ,  0.58795 ,
       -0.55999 ,  0.63368 , -0.043983, -0.68731 , -0.37841 ,  0.38026 ,
        0.61641 , -0.88269 , -0.12346 , -0.37928 , -0.38318 ,  0.23868 ,
        0.6685  , -0.43321 , -0.11065 ,  0.081723,  1.1569  ,  0.78958 ,
       -0.21223 , -2.3211  , -0.67806 ,  0.44561 ,  0.65707 ,  0.1045  ,
        0.46217 ,  0.19912 ,  0.25802 ,  0.057194,  0.53443 , -0.43133 ,
       -0.34311 ,  0.59789 , -0.58417 ,  0.068995,  0.23944 , -0.85181 ,
        0.30379 , -0.34177 , -0.25746 , -0.031101, -0.16285 ,  0.45169 ,
       -0.91627 ,  0.64521 ,  0.73281 , -0.22752 , 

For French word vectors, we're using those from http://fauconnier.github.io/index.html

In [17]:
w2v_path='data/datasets/nlp/frWac_non_lem_no_postag_no_phrase_200_skip_cut100.bin'
# fr_model = word2vec.Word2Vec.load_word2vec_format(w2v_path, binary=True)
fr_model = gensim.models.KeyedVectors.load_word2vec_format(w2v_path, binary=True)
fr_voc = fr_model.vocab
dim_fr_vec = 200

We need to map each word index in our vocabs to their word vector. Not every word in our vocabs will be in our word vectors, since our tokenization approach won't be identical to the word vector creators - in these cases we simply create a random vector.

In [18]:
def create_emb(w2v, targ_vocab, dim_vec):
    vocab_size = len(targ_vocab)
    emb = np.zeros((vocab_size, dim_vec))
    found=0

    for i, word in enumerate(targ_vocab):
        try: emb[i] = w2v[word]; found+=1
        except KeyError: emb[i] = np.random.normal(scale=0.6, size=(dim_vec,))

    return emb, found

In [19]:
en_embs, found = create_emb(en_w2v, en_vocab, dim_en_vec); en_embs.shape, found

((19549, 100), 17201)

In [20]:
fr_embs, found = create_emb(fr_model, fr_vocab, dim_fr_vec); fr_embs.shape, found

((26709, 200), 21878)

## Prep data

Each sentence has to be of equal length. Keras has a convenient function `pad_sequences` to truncate and/or pad each sentence as required - even although we're not using keras for the neural net, we can still use any functions from it we need!

In [21]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 30
en_padded = pad_sequences(en_ids, maxlen, 'int64', "post", "post")
fr_padded = pad_sequences(fr_ids, maxlen, 'int64', "post", "post")
en_padded.shape, fr_padded.shape, en_embs.shape

((52331, 30), (52331, 30), (19549, 100))

And of course we need to separate our training and test sets...

In [22]:
from sklearn import model_selection
fr_train, fr_test, en_train, en_test = model_selection.train_test_split(
    fr_padded, en_padded, test_size=0.1)

[o.shape for o in (fr_train, fr_test, en_train, en_test)]

[(47097, 30), (5234, 30), (47097, 30), (5234, 30)]

Here's an example of a French and English sentence, after encoding and padding.

In [23]:
fr_train[0], en_train[0]

(array([ 489,   13,    4,   45,  756,    3,  996,    5, 1601,    3,   16,
        6208,   25,    6, 1478,    7,    3,  943,   31,   45,  696,  864,
         391,   25,    6,  312,    2,    0,    0,    0]),
 array([  29,    5,   50, 2002,    6,  401,    3,  743, 1149,    5, 3177,
           3, 1644,   52, 1607, 1280,    7, 2576,   36, 1011,  492,  182,
        1019,    2,    0,    0,    0,    0,    0,    0]))

## Model

### Basic encoder-decoder

In [24]:
def long_t(arr): return Variable(torch.LongTensor(arr)).cuda()

In [25]:
fr_emb_t = torch.FloatTensor(fr_embs).cuda()
en_emb_t = torch.FloatTensor(en_embs).cuda()

In [26]:
def create_emb(emb_mat, non_trainable=False):
    output_size, emb_size = emb_mat.size()
    emb = nn.Embedding(output_size, emb_size)
    emb.load_state_dict({'weight': emb_mat})
    if non_trainable:
        for param in emb.parameters(): 
            param.requires_grad = False
    return emb, emb_size, output_size

Turning a sequence into a representation can be done using an RNN (called the 'encoder'. This approach is useful because RNN's are able to keep track of state and memory, which is obviously important in forming a complete understanding of a sentence.
* `bidirectional=True` passes the original sequence through an RNN, and the reversed sequence through a different RNN and concatenates the results. This allows us to look forward and backwards.
* We do this because in language things that happen later often influence what came before (i.e. in Spanish, "el chico, la chica" means the boy, the girl; the word for "the" is determined by the gender of the subject, which comes after).

In [27]:
class EncoderRNN(nn.Module):
    def __init__(self, embs, hidden_size, n_layers=2):
        super(EncoderRNN, self).__init__()
        self.emb, emb_size, output_size = create_emb(embs, True)
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True, num_layers=n_layers)
#                          ,bidirectional=True)
        
    def forward(self, input, hidden):
        return self.gru(self.emb(input), hidden)

    def initHidden(self, batch_size):
        return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))

In [28]:
def encode(inp, encoder):
    batch_size, input_length = inp.size()
    hidden = encoder.initHidden(batch_size).cuda()
    enc_outputs, hidden = encoder(inp, hidden)
    return long_t([SOS]*batch_size), enc_outputs, hidden    

Finally, we arrive at a vector representation of the sequence which captures everything we need to translate it. We feed this vector into more RNN's, which are trying to generate the labels. After this, we make a classification for what each word is in the output sequence.

In [29]:
class DecoderRNN(nn.Module):
    def __init__(self, embs, hidden_size, n_layers=2):
        super(DecoderRNN, self).__init__()
        self.emb, emb_size, output_size = create_emb(embs)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True, num_layers=n_layers)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, inp, hidden):
        emb = self.emb(inp).unsqueeze(1)
        res, hidden = self.gru(emb, hidden)
        res = F.log_softmax(self.out(res[:,0]), dim=-1)
        return res, hidden

This graph demonstrates the accuracy decay for a neural translation task. With an encoding/decoding technique, larger input sequences result in less accuracy.

<img src="https://smerity.com/media/images/articles/2016/bahdanau_attn.png" width="600">

This can be mitigated using an attentional model.

### Attentional model

In [30]:
def Var(*sz): return Parameter(Arr(*sz)).cuda()

### Remember to Fix this

In [31]:
def Var(*sz): 
    if len(sz) < 2:
        t_k = torch.randn(sz)/math.sqrt(sz[0])
    else:
        t = torch.zeros(sz)
        t_k = torch.nn.init.kaiming_normal(t, a=0, mode='fan_in')
    return torch.nn.Parameter(t_k).cuda()

def Bias(*sz):
    return torch.nn.Parameter(torch.zeros(sz)).cuda()

In [32]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, embs, hidden_size, p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.emb, emb_size, output_size = create_emb(embs)
        self.W1 = Var(hidden_size, hidden_size)
        self.W2 = Var(hidden_size, hidden_size)
        self.W3 = Var(emb_size+hidden_size, hidden_size)
        self.b2 = Bias(hidden_size)
        self.b3 = Bias(hidden_size)
        self.V = Var(hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers=2)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, inp, hidden, enc_outputs):
        emb_inp = self.emb(inp)
        w1e = enc_outputs @ self.W1
        w2h = (hidden[-1] @ self.W2 + self.b2).unsqueeze(1)
        u = F.tanh(w1e + w2h)
        a = (self.V * u).sum(2)
        torch.bmm
        a = F.softmax(a, dim=-1).unsqueeze(2)
        Xa = (a * enc_outputs).sum(1)
        res = torch.cat([emb_inp, Xa.squeeze(1)], 1) @ self.W3
        res = (res + self.b3).unsqueeze(0)
        res, hidden = self.gru(res, hidden)
        res = F.log_softmax(self.out(res.squeeze(0)), dim=-1)
        return res, hidden

In [143]:
# PYTORCH VERSION
class AttnDecoderRNN_P(nn.Module):
    def __init__(self, embs, hidden_size, dropout_p=0.1):
#     def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN_P, self).__init__()
    
        self.embedding, emb_size, output_size = create_emb(embs)
        
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
#         self.max_length = max_length
        self.emb_size = emb_size
#         self.embedding = nn.Embedding(self.output_size, self.hidden_size)
#         self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn = nn.Linear(self.hidden_size * 2, self.emb_size)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input)
#         embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        print(embedded.shape)
        print(embedded.view(1,1,-1).shape)
        print(hidden[0].shape)
        print(torch.cat((embedded[0], hidden[0]), 1).shape)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

### Attention testing

Pytorch makes it easy to check intermediate results, when creating a custom architecture such as this one, since you can interactively run each function.

In [33]:
def get_batch(x, y, batch_size=16):
    idxs = np.random.permutation(len(x))[:batch_size]
    return x[idxs], y[idxs]

In [34]:
hidden_size = 128
fra, eng = get_batch(fr_train, en_train, 4)
inp = long_t(fra)
targ = long_t(eng)
emb, emb_size, output_size = create_emb(en_emb_t)
emb.cuda()
inp.size()

torch.Size([4, 30])

In [69]:
W1 = Var(hidden_size, hidden_size)
W2 = Var(hidden_size, hidden_size)
W3 = Var(emb_size+hidden_size, hidden_size)
b2 = Bias(hidden_size)
b3 = Bias(hidden_size)
V = Var(hidden_size)
gru = nn.GRU(hidden_size, hidden_size, num_layers=2).cuda()
out = nn.Linear(hidden_size, output_size).cuda()

In [70]:
dec_inputs, enc_outputs, hidden = encode(inp, encoder)
enc_outputs.size(), hidden.size()

(torch.Size([4, 30, 128]), torch.Size([2, 4, 128]))

In [71]:
emb_inp = emb(dec_inputs); emb_inp.size()

torch.Size([4, 100])

In [72]:
# w1e = W1 @ h(i)
# [4, 30, 128] @ [128, 128] = [4, 30, 128]
w1e = enc_outputs @ W1; w1e.size()

torch.Size([4, 30, 128])

In [97]:
hidden.shape

torch.Size([2, 4, 128])

In [103]:
# w2h = W2 @ d(t)
# d(t_steps) = [2, 4, 128]
# d[-1] = [4, 128]
# [4, 128] @ [128, 128] = [4, 128]
# unsqueeze = [4, 1, 128]
w2h = (hidden[-1] @ W2 + b2).unsqueeze(1); w2h.size()
# print(w2h.shape)
# print(b2.shape)
# w2h = (w2h+b2.expand_as(w2h)).unsqueeze(1); w2h.size()

torch.Size([4, 1, 128])

In [106]:
F.tanh(w1e + w2h).shape

torch.Size([4, 30, 128])

In [107]:
V.shape

torch.Size([128])

In [109]:
(u @ V).shape

torch.Size([4, 30])

In [110]:
F.softmax(a, dim=-1).shape

torch.Size([4, 30, 1])

In [111]:
F.softmax(a, dim=-1).unsqueeze(2).shape

torch.Size([4, 30, 1, 1])

In [108]:
# [4, 30, 128] + [4, 30, 128] = [4, 30, 128]
u = F.tanh(w1e + w2h)

# [4, 30, 128] @ [128] = [4, 30]
a = (u @ V)
# a = (V.expand_as(u)*u).sum(2).squeeze(2)


a = F.softmax(a, dim=-1).unsqueeze(2); a.size(),a.sum(1).squeeze(1)

(torch.Size([4, 30, 1]), Variable containing:
  1.0000
  1.0000
  1.0000
  1.0000
 [torch.cuda.FloatTensor of size 4 (GPU 0)])

In [113]:
a.shape

torch.Size([4, 30, 1])

In [117]:
print(enc_outputs.shape)

torch.Size([4, 30, 128])


In [119]:
a @ enc_outputs

RuntimeError: invalid argument 6: wrong matrix size at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/THC/generic/THCTensorMathBlas.cu:467

In [115]:

Xa = (a.expand_as(enc_outputs) * enc_outputs).sum(1); Xa.size()

torch.Size([4, 128])

In [ ]:
res = torch.cat([emb_inp, Xa.squeeze(1)], 1) @ W3
res = (res+b3.expand_as(res)).unsqueeze(0); res.size()

In [ ]:
res, hidden = gru(res, hidden); res.size(), hidden.size()

In [ ]:
res = F.log_softmax(out(res.squeeze(0)), dim=-1); res.size()

## Train

Pytorch has limited functionality for training models automatically - you will generally have to write your own training loops. However, Pytorch makes it far easier to customize how this training is done, such as using *teacher forcing*.

In [138]:
def train(inp, targ, encoder, decoder, enc_opt, dec_opt, crit):
    decoder_input, encoder_outputs, hidden = encode(inp, encoder)
    target_length = targ.size()[1]
    
    enc_opt.zero_grad(); dec_opt.zero_grad()
    loss = 0

    for di in range(target_length):
        decoder_output, hidden = decoder(decoder_input, hidden, encoder_outputs)
        decoder_input = targ[:, di]
        loss += crit(decoder_output, decoder_input)

    loss.backward()
    enc_opt.step(); dec_opt.step()
    return loss.data[0] / target_length

In [139]:
def req_grad_params(o):
    return (p for p in o.parameters() if p.requires_grad)

In [140]:
def trainEpochs(encoder, decoder, n_epochs, print_every=1000, lr=0.01):
    loss_total = 0 # Reset every print_every
    
    enc_opt = optim.RMSprop(req_grad_params(encoder), lr=lr)
    dec_opt = optim.RMSprop(decoder.parameters(), lr=lr)
    crit = nn.NLLLoss().cuda()
    
    for epoch in range(n_epochs):
        fra, eng = get_batch(fr_train, en_train, 64)
        inp = long_t(fra)
        targ = long_t(eng)
        loss = train(inp, targ, encoder, decoder, enc_opt, dec_opt, crit)
        loss_total += loss

        if epoch % print_every == print_every-1:
            print('%d %d%% %.4f' % (epoch, epoch / n_epochs * 100, loss_total / print_every))
            loss_total = 0

## Run

In [144]:
hidden_size = 128
encoder = EncoderRNN(fr_emb_t, hidden_size).cuda()
decoder = AttnDecoderRNN_P(en_emb_t, hidden_size).cuda()

In [145]:
trainEpochs(encoder, decoder, 10000, print_every=500, lr=0.005)

torch.Size([64, 100])
torch.Size([1, 1, 6400])
torch.Size([64, 128])


RuntimeError: dimension out of range (expected to be in range of [-1, 0], but got 1)

## Testing

In [ ]:
def evaluate(inp):
    decoder_input, encoder_outputs, hidden = encode(inp, encoder)
    target_length = maxlen

    decoded_words = []
    for di in range(target_length):
        decoder_output, hidden = decoder(decoder_input, hidden, encoder_outputs)
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if ni==PAD: break
        decoded_words.append(en_vocab[ni])
        decoder_input = long_t([ni])
    
    return decoded_words

In [ ]:
def sent2ids(sent):
    ids = [fr_w2id[t] for t in simple_toks(sent)]
    return pad_sequences([ids], maxlen, 'int64', "post", "post")

In [ ]:
def fr2en(sent): 
    ids = long_t(sent2ids(sent))
    trans = evaluate(ids)
    return ' '.join(trans)

In [ ]:
i=8
print(en_qs[i],fr_qs[i])
fr2en(fr_qs[i])